In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [97]:
df_binance_futures = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\df_binance_futures.csv')
df_binance_futures = df_binance_futures.assign(exchange_id = 'binance_futures')
df_binance_futures.head()

,timestamp,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,2022-10-01 00:00:00+00:00,1664582400000,5.406,5.406,5.401,5.403,20601.0,binance_futures,2022-10-02 00:17:30.835182+00:00,APE_USDT,2022,10
1,2022-10-01 00:01:00+00:00,1664582460000,5.403,5.404,5.401,5.403,7417.0,binance_futures,2022-10-02 00:17:30.835182+00:00,APE_USDT,2022,10
2,2022-10-01 00:02:00+00:00,1664582520000,5.403,5.407,5.403,5.407,4555.0,binance_futures,2022-10-02 00:17:30.835182+00:00,APE_USDT,2022,10
3,2022-10-01 00:03:00+00:00,1664582580000,5.406,5.408,5.404,5.405,11194.0,binance_futures,2022-10-02 00:17:30.835182+00:00,APE_USDT,2022,10
4,2022-10-01 00:04:00+00:00,1664582640000,5.404,5.406,5.404,5.405,1942.0,binance_futures,2022-10-02 00:17:30.835182+00:00,APE_USDT,2022,10


In [98]:
df_binance_spot = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\df_binance_spot.csv')
df_binance_spot = df_binance_spot.assign(exchange_id = 'binance_spot')
df_binance_spot.head()

,timestamp,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,2022-10-01 00:00:00+00:00,1664582400000,5.408,5.408,5.404,5.404,726.43,binance_spot,2023-02-10 21:53:22.710648+00:00,APE_USDT,2022,10
1,2022-10-01 00:01:00+00:00,1664582460000,5.404,5.406,5.403,5.406,838.73,binance_spot,2023-02-10 21:53:22.710648+00:00,APE_USDT,2022,10
2,2022-10-01 00:02:00+00:00,1664582520000,5.406,5.409,5.406,5.407,745.08,binance_spot,2023-02-10 21:53:22.710648+00:00,APE_USDT,2022,10
3,2022-10-01 00:03:00+00:00,1664582580000,5.407,5.409,5.405,5.406,2627.81,binance_spot,2023-02-10 21:53:22.710648+00:00,APE_USDT,2022,10
4,2022-10-01 00:04:00+00:00,1664582640000,5.405,5.407,5.405,5.407,201.09,binance_spot,2023-02-10 21:53:22.710648+00:00,APE_USDT,2022,10


In [99]:
df_binance_us = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\df_binanceus.csv')
df_binance_us.head()

,timestamp,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,2022-10-01 00:00:00+00:00,1664582400000,5.4265,5.4265,5.4265,5.4265,0.0,binanceus,2023-02-10 21:49:00.544512+00:00,APE_USDT,2022,10
1,2022-10-01 00:01:00+00:00,1664582460000,5.4265,5.4265,5.4265,5.4265,0.0,binanceus,2023-02-10 21:49:00.544512+00:00,APE_USDT,2022,10
2,2022-10-01 00:02:00+00:00,1664582520000,5.4265,5.4265,5.4265,5.4265,0.0,binanceus,2023-02-10 21:49:00.544512+00:00,APE_USDT,2022,10
3,2022-10-01 00:03:00+00:00,1664582580000,5.4265,5.4265,5.4265,5.4265,0.0,binanceus,2023-02-10 21:49:00.544512+00:00,APE_USDT,2022,10
4,2022-10-01 00:04:00+00:00,1664582640000,5.4265,5.4265,5.4265,5.4265,0.0,binanceus,2023-02-10 21:49:00.544512+00:00,APE_USDT,2022,10


In [2]:
df_okx = pd.read_csv(r'C:\Users\James Zhang\Desktop\sorrentum\sorrentum_sandbox\projects\SorrIssue2_Cross_exchange_arbitrage_CEX_CEX\df_okx.csv')
df_okx.head()

,timestamp,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,2022-10-01 15:00:00+00:00,1664636400000,5.3401,5.3401,5.3361,5.3361,189.878030,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
1,2022-10-01 15:01:00+00:00,1664636460000,5.3356,5.3356,5.3262,5.3262,1404.094091,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
2,2022-10-01 15:02:00+00:00,1664636520000,5.3273,5.3318,5.3273,5.3313,1525.072300,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
3,2022-10-01 15:03:00+00:00,1664636580000,5.3316,5.3379,5.3316,5.3354,530.874701,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
4,2022-10-01 15:04:00+00:00,1664636640000,5.3348,5.3348,5.3303,5.3303,259.569769,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10


let's work with a small sample size of data: october 2022 in okx exchange, and once we figure out how to do stuff, then we can scale upwards

In [7]:
temp = df_okx.loc[((df_okx['year'] == 2022) & (df_okx['month'] == 10))]
temp.head()

,timestamp,timestamp.1,open,high,low,close,volume,exchange_id,knowledge_timestamp,currency_pair,year,month
0,2022-10-01 15:00:00+00:00,1664636400000,5.3401,5.3401,5.3361,5.3361,189.878030,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
1,2022-10-01 15:01:00+00:00,1664636460000,5.3356,5.3356,5.3262,5.3262,1404.094091,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
2,2022-10-01 15:02:00+00:00,1664636520000,5.3273,5.3318,5.3273,5.3313,1525.072300,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
3,2022-10-01 15:03:00+00:00,1664636580000,5.3316,5.3379,5.3316,5.3354,530.874701,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10
4,2022-10-01 15:04:00+00:00,1664636640000,5.3348,5.3348,5.3303,5.3303,259.569769,okx,2023-02-11 12:46:52.784687+00:00,APE_USDT,2022,10


In [8]:
temp.drop(columns=['timestamp.1', 'knowledge_timestamp', 'open', 'close', 'year', 'month', 'exchange_id'], inplace=True)

In [9]:
grouped = temp.groupby('currency_pair')
dfs = [grouped.get_group(x) for x in grouped.groups]

In [10]:
merged_okx = dfs[0]['timestamp'] # save this for later, will help with merging
timestamp = dfs[0]['timestamp']

In [11]:
for df in dfs:
    df['vwap-okx::'+df['currency_pair'].unique()[0]] = np.cumsum(df['volume'] * (df['high'] + df['low'])/2) / np.cumsum(df['volume'])
    df.rename(columns={"volume" : 'volume-okx::'+df['currency_pair'].unique()[0]}, inplace=True)
    df.drop(columns=['high', 'low', 'currency_pair'], inplace=True)
    df.set_index('timestamp', inplace=True)

it looks like the above was done correctly -> now we have to try and merge all of them

In [12]:
for df in dfs:
    merged_okx = pd.merge(merged_okx, df, left_on='timestamp', right_on='timestamp')

In [13]:
merged_okx = merged_okx.set_index('timestamp')
merged_okx

,volume-okx::APE_USDT,vwap-okx::APE_USDT,volume-okx::AVAX_USDT,vwap-okx::AVAX_USDT,volume-okx::AXS_USDT,vwap-okx::AXS_USDT,volume-okx::BTC_USDT,vwap-okx::BTC_USDT,volume-okx::DOGE_USDT,vwap-okx::DOGE_USDT,...,volume-okx::LINK_USDT,vwap-okx::LINK_USDT,volume-okx::NEAR_USDT,vwap-okx::NEAR_USDT,volume-okx::SAND_USDT,vwap-okx::SAND_USDT,volume-okx::SOL_USDT,vwap-okx::SOL_USDT,volume-okx::XRP_USDT,vwap-okx::XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-10-01 15:00:00+00:00,189.878030,5.338100,28.726003,17.185000,17.767561,12.640000,0.249802,19321.050000,1.558154e+04,0.061460,...,734.434521,7.520000,2079.172278,3.555500,1049.994113,0.837650,1.371908,32.890000,8668.530595,0.476615
2022-10-01 15:01:00+00:00,1404.094091,5.331758,64.657472,17.178076,2.010417,12.640000,7.931582,19315.136251,1.224445e+05,0.061398,...,546.093082,7.517868,5514.181050,3.553685,4191.800000,0.837330,673.490763,32.855071,45756.333858,0.476493
2022-10-01 15:02:00+00:00,1525.072300,5.330678,0.011905,17.178074,0.000000,12.640000,0.372186,19314.864897,3.682257e+04,0.061389,...,6.357715,7.517846,59.287817,3.553656,520.495735,0.837264,208.113192,32.853876,36390.661517,0.476490
2022-10-01 15:03:00+00:00,530.874701,5.331270,26.644861,17.176282,0.178015,12.640089,2.130541,19315.440221,2.759960e+04,0.061391,...,201.010928,7.518137,983.510440,3.553524,1930.253027,0.837248,38.984093,32.854558,32015.369617,0.476651
2022-10-01 15:04:00+00:00,259.569769,5.331355,40.827680,17.173418,0.000000,12.640089,1.153486,19315.787095,6.177918e+04,0.061386,...,82.386709,7.518209,2128.638720,3.552827,4378.745562,0.836977,42.055153,32.855231,111647.561655,0.477084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-31 13:15:00+00:00,2220.890405,4.753914,31.467091,16.981017,161.662144,9.672030,0.297364,19681.670574,1.332704e+06,0.093484,...,681.567466,7.223080,1689.795082,3.097338,24316.110275,0.795820,76.328451,31.080670,51207.442613,0.476448
2022-10-31 13:16:00+00:00,490.571188,4.753923,161.070076,16.981798,54.342640,9.671984,0.184068,19681.678820,7.392846e+05,0.093500,...,751.366484,7.223396,1659.744407,3.097357,4352.069129,0.795835,30.438649,31.080780,54935.098462,0.476444
2022-10-31 13:17:00+00:00,598.120377,4.753934,489.361336,16.984164,236.596097,9.671783,0.619490,19681.706612,9.256884e+05,0.093520,...,508.607888,7.223611,475.013401,3.097363,8312.630257,0.795863,165.018809,31.081377,62034.089256,0.476440


In [56]:
def create_single(exchange_df):
    exchange_df = exchange_df.sort_values(by='timestamp')
    id = exchange_df['exchange_id'].unique()[0]
    exchange_df = exchange_df.drop(columns=['timestamp.1', 'knowledge_timestamp', 'open', 'close', 'year', 'month', 'exchange_id'])
    grouped = exchange_df.groupby('currency_pair')
    dfs = [grouped.get_group(x) for x in grouped.groups]
    merged = dfs[0]['timestamp'] # save this for later, will help with merging
    for df in dfs:
        df[f"vwap-{id}:{df['currency_pair'].unique()[0]}"] = np.cumsum(df['volume'] * (df['high'] + df['low'])/2) / np.cumsum(df['volume'])
        df.rename(columns={'volume' : 'volume-' + id +':'+df['currency_pair'].unique()[0]}, inplace=True)
        df.drop(columns=['high', 'low', 'currency_pair'], inplace=True)
        df.set_index('timestamp', inplace=True)
    for df in dfs:
        merged = pd.merge(merged, df, left_on='timestamp', right_on='timestamp')
    timestamp = merged['timestamp']
    merged = merged.set_index('timestamp')
    return merged, timestamp

In [57]:
single, t = create_single(df_okx)
single

,volume-okx:APE_USDT,vwap-okx:APE_USDT,volume-okx:AVAX_USDT,vwap-okx:AVAX_USDT,volume-okx:AXS_USDT,vwap-okx:AXS_USDT,volume-okx:BNB_USDT,vwap-okx:BNB_USDT,volume-okx:BTC_USDT,vwap-okx:BTC_USDT,...,volume-okx:SAND_USDT,vwap-okx:SAND_USDT,volume-okx:SOL_USDT,vwap-okx:SOL_USDT,volume-okx:STORJ_USDT,vwap-okx:STORJ_USDT,volume-okx:WAVES_USDT,vwap-okx:WAVES_USDT,volume-okx:XRP_USDT,vwap-okx:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-01 15:00:00+00:00,143.559420,7.722025,10.223912,46.340432,5.745520,60.458417,1.409575,245.395919,0.325563,38475.039256,...,7415.647705,1.519536,165.079423,58.248381,4108.159768,1.358559,55.988844,15.465800,43952.102195,0.680726
2023-01-01 15:01:00+00:00,0.000000,7.722025,0.958191,46.340432,0.000000,60.458417,3.170915,245.395628,0.032026,38475.038552,...,537.285475,1.519535,32.146558,58.248369,0.000000,1.358559,0.000000,15.465800,15153.917539,0.680725
2023-01-01 15:02:00+00:00,0.000000,7.722025,0.010050,46.340432,0.000000,60.458417,3.585290,245.395330,0.279657,38475.032408,...,52.963882,1.519535,190.368535,58.248303,1412.705914,1.358555,0.000000,15.465800,28369.763341,0.680724
2023-01-01 15:03:00+00:00,68.454178,7.722021,0.010238,46.340432,0.000000,60.458417,7.773471,245.394697,0.252629,38475.026858,...,214.459606,1.519535,9.205430,58.248300,0.000000,1.358555,0.000000,15.465800,5.398469,0.680724
2023-01-01 15:04:00+00:00,319.673310,7.722005,20.230825,46.340424,0.000000,60.458417,3.543093,245.394385,0.088499,38475.024914,...,795.732658,1.519534,269.418799,58.248206,263.065776,1.358554,177.720696,15.465772,115470.274134,0.680717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 13:15:00+00:00,748.782823,7.517841,13.863603,45.890622,0.000000,58.765350,6.830231,276.040637,0.657629,38021.177807,...,2946.396633,1.480779,280.840973,56.795526,54.215121,1.338541,921.038778,14.483248,18600.867465,0.667274
2023-01-31 13:16:00+00:00,1711.776069,7.517808,16.577327,45.890617,45.360064,58.765264,1.091403,276.041513,0.857525,38021.165182,...,6191.762224,1.480774,455.824680,56.795422,571.241028,1.338539,544.728049,14.483180,22391.441465,0.667273
2023-01-31 13:17:00+00:00,512.456285,7.517797,1.870275,45.890617,4.759460,58.765255,2.238450,276.043310,3.007150,38021.120921,...,1389.853643,1.480773,239.935366,56.795367,141.850043,1.338539,1961.725418,14.482937,22905.081298,0.667272


# can we get this as a multi-index df
in fact, we will use this normal df and convert it into multiindex

In [61]:
vols = ""
vol_cols = list(merged_okx.loc[:, merged_okx.columns.str.startswith('volume')].columns)
for elem in vol_cols:
    vols = "".join([vols, elem]) + " "
vols

'volume-okx::APE_USDT volume-okx::AVAX_USDT volume-okx::AXS_USDT volume-okx::BTC_USDT volume-okx::DOGE_USDT volume-okx::DOT_USDT volume-okx::ETH_USDT volume-okx::FTM_USDT volume-okx::GMT_USDT volume-okx::LINK_USDT volume-okx::NEAR_USDT volume-okx::SAND_USDT volume-okx::SOL_USDT volume-okx::XRP_USDT '

In [62]:
vwap = ""
vwap_cols = list(merged_okx.loc[:, merged_okx.columns.str.startswith('vwap')].columns)
for elem in vwap_cols:
    vwap = "".join([vwap, elem]) + " "
len(vwap.split())

14

In [63]:
volume_str = "volume " * 14
vwap_str = "vwap " * 14
exchange_str = "okx " * 28

In [64]:
multi = pd.DataFrame(np.array(merged_okx), columns=[(volume_str + vwap_str).split(), exchange_str.split(), (vols + vwap).split()])
multi.set_index(timestamp, inplace=True)

In [65]:
multi

volume                        \
                                           okx                         
                          volume-okx::APE_USDT volume-okx::AVAX_USDT   
timestamp                                                              
2022-10-01 15:00:00+00:00           189.878030              5.338100   
2022-10-01 15:01:00+00:00          1404.094091              5.331758   
2022-10-01 15:02:00+00:00          1525.072300              5.330678   
2022-10-01 15:03:00+00:00           530.874701              5.331270   
2022-10-01 15:04:00+00:00           259.569769              5.331355   
...                                        ...                   ...   
2022-10-31 13:15:00+00:00          2220.890405              4.753914   
2022-10-31 13:16:00+00:00           490.571188              4.753923   
2022-10-31 13:17:00+00:00           598.120377              4.753934   
2022-10-31 13:18:00+00:00           379.773821              4.753940   
2022-10-31 13:19:00+00:00            59.192317              4.753942   

                                                                     \
                                                                      
                          volume-okx::AXS_USDT volume-okx::BTC_USDT   
timestamp                                                             
2022-10-01 15:00:00+00:00            28.726003            17.185000   
2022-10-01 15:01:00+00:00            64.657472            17.178076   
2022-10-01 15:02:00+00:00             0.011905            17.178074   
2022-10-01 15:03:00+00:00            26.644861            17.176282   
2022-10-01 15:04:00+00:00            40.827680            17.173418   
...                                        ...                  ...   
2022-10-31 13:15:00+00:00            31.467091            16.981017   
2022-10-31 13:16:00+00:00           161.070076            16.981798   
2022-10-31 13:17:00+00:00           489.361336            16.984164   
2022-10-31 13:18:00+00:00            31.799362            16.984317   
2022-10-31 13:19:00+00:00            82.840578            16.984714   

                                                                      \
                                                                       
                          volume-okx::DOGE_USDT volume-okx::DOT_USDT   
timestamp                                                              
2022-10-01 15:00:00+00:00             17.767561            12.640000   
2022-10-01 15:01:00+00:00              2.010417            12.640000   
2022-10-01 15:02:00+00:00              0.000000            12.640000   
2022-10-01 15:03:00+00:00              0.178015            12.640089   
2022-10-01 15:04:00+00:00              0.000000            12.640089   
...                                         ...                  ...   
2022-10-31 13:15:00+00:00            161.662144             9.672030   
2022-10-31 13:16:00+00:00             54.342640             9.671984   
2022-10-31 13:17:00+00:00            236.596097             9.671783   
2022-10-31 13:18:00+00:00            754.739301             9.671153   
2022-10-31 13:19:00+00:00             36.434347             9.671122   

                                                                     \
                                                                      
                          volume-okx::ETH_USDT volume-okx::FTM_USDT   
timestamp                                                             
2022-10-01 15:00:00+00:00             0.249802         19321.050000   
2022-10-01 15:01:00+00:00             7.931582         19315.136251   
2022-10-01 15:02:00+00:00             0.372186         19314.864897   
2022-10-01 15:03:00+00:00             2.130541         19315.440221   
2022-10-01 15:04:00+00:00             1.153486         19315.787095   
...                                        ...                  ...   
2022-10-31 13:15:00+00:00             0.297364         19681.670574   
2022-10-31 13:16:00+00:00      

In [68]:
d = dict(zip(multi.columns.levels[2], [word[2][word[2].rfind(':') + 1:] for word in multi.columns]))
multi = multi.rename(columns=d, level=2)
multi

volume                                   \
                                   okx                                    
                              APE_USDT AVAX_USDT    AXS_USDT   BTC_USDT   
timestamp                                                                 
2022-10-01 15:00:00+00:00   189.878030  5.338100   28.726003  17.185000   
2022-10-01 15:01:00+00:00  1404.094091  5.331758   64.657472  17.178076   
2022-10-01 15:02:00+00:00  1525.072300  5.330678    0.011905  17.178074   
2022-10-01 15:03:00+00:00   530.874701  5.331270   26.644861  17.176282   
2022-10-01 15:04:00+00:00   259.569769  5.331355   40.827680  17.173418   
...                                ...       ...         ...        ...   
2022-10-31 13:15:00+00:00  2220.890405  4.753914   31.467091  16.981017   
2022-10-31 13:16:00+00:00   490.571188  4.753923  161.070076  16.981798   
2022-10-31 13:17:00+00:00   598.120377  4.753934  489.361336  16.984164   
2022-10-31 13:18:00+00:00   379.773821  4.753940   31.799362  16.984317   
2022-10-31 13:19:00+00:00    59.192317  4.753942   82.840578  16.984714   

                                                                          \
                                                                           
                            DOGE_USDT   DOT_USDT  ETH_USDT      FTM_USDT   
timestamp                                                                  
2022-10-01 15:00:00+00:00   17.767561  12.640000  0.249802  19321.050000   
2022-10-01 15:01:00+00:00    2.010417  12.640000  7.931582  19315.136251   
2022-10-01 15:02:00+00:00    0.000000  12.640000  0.372186  19314.864897   
2022-10-01 15:03:00+00:00    0.178015  12.640089  2.130541  19315.440221   
2022-10-01 15:04:00+00:00    0.000000  12.640089  1.153486  19315.787095   
...                               ...        ...       ...           ...   
2022-10-31 13:15:00+00:00  161.662144   9.672030  0.297364  19681.670574   
2022-10-31 13:16:00+00:00   54.342640   9.671984  0.184068  19681.678820   
2022-10-31 13:17:00+00:00  236.596097   9.671783  0.619490  19681.706612   
2022-10-31 13:18:00+00:00  754.739301   9.671153  0.482864  19681.727995   
2022-10-31 13:19:00+00:00   36.434347   9.671122  2.915668  19681.856968   

                                                   ...          vwap  \
                                                   ...           okx   
                               GMT_USDT LINK_USDT  ...     DOGE_USDT   
timestamp                                          ...                 
2022-10-01 15:00:00+00:00  1.558154e+04  0.061460  ...    734.434521   
2022-10-01 15:01:00+00:00  1.224445e+05  0.061398  ...    546.093082   
2022-10-01 15:02:00+00:00  3.682257e+04  0.061389  ...      6.357715   
2022-10-01 15:03:00+00:00  2.759960e+04  0.061391  ...    201.010928   
2022-10-01 15:04:00+00:00  6.177918e+04  0.061386  ...     82.386709   
...                                 ...       ...  ...           ...   
2022-10-31 13:15:00+00:00  1.332704e+06  0.093484  ...    681.567466   
2022-10-31 13:16:00+00:00  7.392846e+05  0.093500  ...    751.366484   
2022-10-31 13:17:00+00:00  9.256884e+05  0.093520  ...    508.607888   
2022-10-31 13:18:00+00:00  5.782954e+05  0.093533  ...    507.827656   
2022-10-31 13:19:00+00:00  9.102564e+05  0.093552  ...  18850.310468   

                                                                          \
                                                                           
                           DOT_USDT     ETH_USDT  FTM_USDT      GMT_USDT   
timestamp                                                                  
2022-10-01 15:00:00+00:00  7.520000  2079.172278  3.555500   1049.994113   
2022-10-01 15:01:00+00:00  7.517868  5514.181050  3.553685   4191.800000   
2022-10-01 15:02:00+00:00  7.517846    59.287817  3.553656    520.495735   
2022-10-01 15:03:00+00:00  7.518137   983.510440  3.553524   1930.253027   
2022-10-01 15:04:00+00:00  7.518209  2128.638720  3.552827   4378.745562   


### the above is the correct df for all coins that exist on the okx exchange in october 2022

In [91]:
def convert_to_multi(df, timestamp):
    # creating most inner column 
    vols, vwap = "", ""
    vol_cols, vwap_cols = list(df.loc[:, df.columns.str.startswith('volume')].columns), list(df.loc[:, df.columns.str.startswith('vwap')].columns)
    for elem in vol_cols:
        vols = "".join([vols, elem]) + " "
    for elem in vwap_cols:
        vwap = "".join([vwap, elem]) + " "

    # creating the middle column (the exchange: okx, binance_spot, binance_future, binanceus)
    exchange_str = ""
    for col in vol_cols:
        h, s = col.rfind('-') + 1, col.rfind(':')
        exchange_str += col[h:s] + " "
    for col in vwap_cols:
        h, s = col.rfind('-') + 1, col.rfind(':')
        exchange_str += col[h:s] + " "

    # creating outer column (the feature: volume or vwap)
    volume_str, vwap_str = "volume " * int(len(list(df.columns))/2), "vwap " * int(len(list(df.columns))/2)

    new_df = pd.DataFrame(np.array(df), columns=[(volume_str + vwap_str).split(), exchange_str.split(), (vols + vwap).split()]) # this line needs to include a new level
    # new_df.set_index(timestamp, inplace=True)

    d = dict(zip(new_df.columns.levels[2], [word[2][word[2].rfind(':') + 1:] for word in new_df.columns]))
    new_df = new_df.rename(columns=d, level=2)
    new_df = new_df.set_index(timestamp)
    return new_df

In [92]:
single_okx, t_okx = create_single(df_okx)
single_okx.head()

,volume-okx:APE_USDT,vwap-okx:APE_USDT,volume-okx:AVAX_USDT,vwap-okx:AVAX_USDT,volume-okx:AXS_USDT,vwap-okx:AXS_USDT,volume-okx:BNB_USDT,vwap-okx:BNB_USDT,volume-okx:BTC_USDT,vwap-okx:BTC_USDT,...,volume-okx:SAND_USDT,vwap-okx:SAND_USDT,volume-okx:SOL_USDT,vwap-okx:SOL_USDT,volume-okx:STORJ_USDT,vwap-okx:STORJ_USDT,volume-okx:WAVES_USDT,vwap-okx:WAVES_USDT,volume-okx:XRP_USDT,vwap-okx:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-01-01 15:00:00+00:00,143.559420,7.722025,10.223912,46.340432,5.74552,60.458417,1.409575,245.395919,0.325563,38475.039256,...,7415.647705,1.519536,165.079423,58.248381,4108.159768,1.358559,55.988844,15.465800,43952.102195,0.680726
2023-01-01 15:01:00+00:00,0.000000,7.722025,0.958191,46.340432,0.00000,60.458417,3.170915,245.395628,0.032026,38475.038552,...,537.285475,1.519535,32.146558,58.248369,0.000000,1.358559,0.000000,15.465800,15153.917539,0.680725
2023-01-01 15:02:00+00:00,0.000000,7.722025,0.010050,46.340432,0.00000,60.458417,3.585290,245.395330,0.279657,38475.032408,...,52.963882,1.519535,190.368535,58.248303,1412.705914,1.358555,0.000000,15.465800,28369.763341,0.680724
2023-01-01 15:03:00+00:00,68.454178,7.722021,0.010238,46.340432,0.00000,60.458417,7.773471,245.394697,0.252629,38475.026858,...,214.459606,1.519535,9.205430,58.248300,0.000000,1.358555,0.000000,15.465800,5.398469,0.680724
2023-01-01 15:04:00+00:00,319.673310,7.722005,20.230825,46.340424,0.00000,60.458417,3.543093,245.394385,0.088499,38475.024914,...,795.732658,1.519534,269.418799,58.248206,263.065776,1.358554,177.720696,15.465772,115470.274134,0.680717


In [94]:
multidf_okx = convert_to_multi(single_okx, t_okx)
multidf_okx

volume                                  \
                                   okx                                   
                              APE_USDT AVAX_USDT   AXS_USDT   BNB_USDT   
timestamp                                                                
2023-01-01 15:00:00+00:00   143.559420  7.722025  10.223912  46.340432   
2023-01-01 15:01:00+00:00     0.000000  7.722025   0.958191  46.340432   
2023-01-01 15:02:00+00:00     0.000000  7.722025   0.010050  46.340432   
2023-01-01 15:03:00+00:00    68.454178  7.722021   0.010238  46.340432   
2023-01-01 15:04:00+00:00   319.673310  7.722005  20.230825  46.340424   
...                                ...       ...        ...        ...   
2023-01-31 13:15:00+00:00   748.782823  7.517841  13.863603  45.890622   
2023-01-31 13:16:00+00:00  1711.776069  7.517808  16.577327  45.890617   
2023-01-31 13:17:00+00:00   512.456285  7.517797   1.870275  45.890617   
2023-01-31 13:18:00+00:00  1348.665222  7.517771   2.912640  45.890616   
2023-01-31 13:19:00+00:00  2379.821350  7.517724  14.915358  45.890612   

                                                                         \
                                                                          
                             BTC_USDT  DOGE_USDT   DOT_USDT    ETH_USDT   
timestamp                                                                 
2023-01-01 15:00:00+00:00    5.745520  60.458417   1.409575  245.395919   
2023-01-01 15:01:00+00:00    0.000000  60.458417   3.170915  245.395628   
2023-01-01 15:02:00+00:00    0.000000  60.458417   3.585290  245.395330   
2023-01-01 15:03:00+00:00    0.000000  60.458417   7.773471  245.394697   
2023-01-01 15:04:00+00:00    0.000000  60.458417   3.543093  245.394385   
...                               ...        ...        ...         ...   
2023-01-31 13:15:00+00:00    0.000000  58.765350   6.830231  276.040637   
2023-01-31 13:16:00+00:00   45.360064  58.765264   1.091403  276.041513   
2023-01-31 13:17:00+00:00    4.759460  58.765255   2.238450  276.043310   
2023-01-31 13:18:00+00:00   60.319632  58.765140   1.299594  276.044354   
2023-01-31 13:19:00+00:00  213.522092  58.764734  19.849352  276.060367   

                                                    ...         vwap  \
                                                    ...          okx   
                            FTM_USDT      GMT_USDT  ...     FTM_USDT   
timestamp                                           ...                
2023-01-01 15:00:00+00:00   0.325563  38475.039256  ...  7415.647705   
2023-01-01 15:01:00+00:00   0.032026  38475.038552  ...   537.285475   
2023-01-01 15:02:00+00:00   0.279657  38475.032408  ...    52.963882   
2023-01-01 15:03:00+00:00   0.252629  38475.026858  ...   214.459606   
2023-01-01 15:04:00+00:00   0.088499  38475.024914  ...   795.732658   
...                              ...           ...  ...          ...   
2023-01-31 13:15:00+00:00   0.657629  38021.177807  ...  2946.396633   
2023-01-31 13:16:00+00:00   0.857525  38021.165182  ...  6191.762224   
2023-01-31 13:17:00+00:00   3.007150  38021.120921  ...  1389.853643   
2023-01-31 13:18:00+00:00   2.210284  38021.088400  ...   501.965245   
2023-01-31 13:19:00+00:00  22.001453  38020.764852  ...  5087.802162   

                                                                         \
                                                                          
                           GMT_USDT   LINK_USDT MATIC_USDT    NEAR_USDT   
timestamp                                                                 
2023-01-01 15:00:00+00:00  1.519536  165.079423  58.248381  4108.159768   
2023-01-01 15:01:00+00:00  1.519535   32.146558  58.248369     0.000000   
2023-01-01 15:02:00+00:00  1.519535  190.368535  58.248303  1412.705914   
2023-01-01 15:03:00+00:00  1.519535    9.205430  58.248300     0.000000   
2023-01-01 15:04:00+00:00  1.519534  269.418799  58.248206   263.065776   
...                             ...    

# while the multiindexing is done correctly, we run into a problem... 
each coin does not exist on the exchange for the same time interval so what is the best way to create this merged df for a single exchange?
- should we have NAN values at for some coins at parts of the exchange -> could run into with merging dfs of different length
    - if we do this, we could group by currency, make our vwap calculations, and then when trying to merge grouped dfs this is where we'd run into an issue
- do we just try to find a common interval where all coins in the exchange exist?
    - we could be wasting a lot of data this way

I think the best way is option 1, so let us make some modifications 

In [104]:
def create_single(exchange_df):
    exchange_df['timestamp'] = exchange_df['timestamp'].astype('datetime64[ms]')
    exchange_df = exchange_df.sort_values(by='timestamp')
    id = exchange_df['exchange_id'].unique()[0]
    exchange_df = exchange_df.drop(columns=['timestamp.1', 'knowledge_timestamp', 'open', 'close', 'year', 'month', 'exchange_id'])
    grouped = exchange_df.groupby('currency_pair')
    dfs = [grouped.get_group(x) for x in grouped.groups]
    timestamp = pd.DataFrame(exchange_df['timestamp'].unique())
    timestamp = timestamp.rename(columns={0:"timestamp"})
    merged = timestamp.copy()
    for df in dfs:
        df[f"vwap-{id}:{df['currency_pair'].unique()[0]}"] = np.cumsum(df['volume'] * (df['high'] + df['low'])/2) / np.cumsum(df['volume'])
        df.rename(columns={'volume' : 'volume-' + id +':'+df['currency_pair'].unique()[0]}, inplace=True)
        df.drop(columns=['high', 'low', 'currency_pair'], inplace=True)
        df.set_index('timestamp', inplace=True)
    for df in dfs:
        merged = pd.merge_asof(merged, df, on='timestamp')
    merged = merged.set_index('timestamp')
    return merged, timestamp

# def convert_to_multi(df, timestamp):
#     vols, vwap = "", ""
#     vol_cols, vwap_cols = list(df.loc[:, df.columns.str.startswith('volume')].columns), list(df.loc[:, df.columns.str.startswith('vwap')].columns)
#     for elem in vol_cols:
#         vols = "".join([vols, elem]) + " "
#     for elem in vwap_cols:
#         vwap = "".join([vwap, elem]) + " "
#     volume_str, vwap_str = "volume " * int(len(list(df.columns))/2), "vwap " * int(len(list(df.columns))/2)
#     new_df = pd.DataFrame(np.array(df), columns=[(volume_str + vwap_str).split(), (vols + vwap).split()])
#     # new_df.set_index(timestamp, inplace=True)
#     rename_dict = dict(zip(new_df.columns.levels[1], [word[1][7:] if word[1].startswith('volume') else word[1][5:] for word in new_df.columns]))
#     new_df = new_df.rename(columns=rename_dict, level=1)
#     new_df = new_df.set_index(timestamp['timestamp'].astype('datetime64[ms]'))
#     return new_df


def convert_to_multi(df, timestamp):
    # creating most inner column 
    vols, vwap = "", ""
    vol_cols, vwap_cols = list(df.loc[:, df.columns.str.startswith('volume')].columns), list(df.loc[:, df.columns.str.startswith('vwap')].columns)
    for elem in vol_cols:
        vols = "".join([vols, elem]) + " "
    for elem in vwap_cols:
        vwap = "".join([vwap, elem]) + " "

    # creating the middle column (the exchange: okx, binance_spot, binance_future, binanceus)
    exchange_str = ""
    for col in vol_cols:
        h, s = col.rfind('-') + 1, col.rfind(':')
        exchange_str += col[h:s] + " "
    for col in vwap_cols:
        h, s = col.rfind('-') + 1, col.rfind(':')
        exchange_str += col[h:s] + " "

    # creating outer column (the feature: volume or vwap)
    volume_str, vwap_str = "volume " * int(len(list(df.columns))/2), "vwap " * int(len(list(df.columns))/2)

    new_df = pd.DataFrame(np.array(df), columns=[(volume_str + vwap_str).split(), exchange_str.split(), (vols + vwap).split()]) # this line needs to include a new level
    # new_df.set_index(timestamp, inplace=True)

    d = dict(zip(new_df.columns.levels[2], [word[2][word[2].rfind(':') + 1:] for word in new_df.columns]))
    new_df = new_df.rename(columns=d, level=2)
    new_df = new_df.set_index(timestamp['timestamp'].astype('datetime64[ms]'))
    return new_df

### okx

In [109]:
single_okx, t_okx = create_single(df_okx)
# single_okx.to_csv('single_okx.csv')
single_okx

,volume-okx:APE_USDT,vwap-okx:APE_USDT,volume-okx:AVAX_USDT,vwap-okx:AVAX_USDT,volume-okx:AXS_USDT,vwap-okx:AXS_USDT,volume-okx:BNB_USDT,vwap-okx:BNB_USDT,volume-okx:BTC_USDT,vwap-okx:BTC_USDT,...,volume-okx:SAND_USDT,vwap-okx:SAND_USDT,volume-okx:SOL_USDT,vwap-okx:SOL_USDT,volume-okx:STORJ_USDT,vwap-okx:STORJ_USDT,volume-okx:WAVES_USDT,vwap-okx:WAVES_USDT,volume-okx:XRP_USDT,vwap-okx:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 15:00:00,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,2.567870,29319.650000,...,NaN,NaN,374.301200,1.642000,0.000000,NaN,3068.481619,6.225000,50610.214000,0.237200
2021-01-01 15:01:00,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,1.985712,29314.678723,...,NaN,NaN,590.031900,1.641388,177.490000,0.299200,1247.744360,6.223844,86664.110000,0.237389
2021-01-01 15:02:00,NaN,NaN,137.169200,3.725500,NaN,NaN,NaN,NaN,35.688282,29282.948554,...,NaN,NaN,954.226900,1.638708,0.000000,0.299200,831.909741,6.222253,238504.613000,0.237936
2021-01-01 15:03:00,NaN,NaN,33.754300,3.724216,NaN,NaN,NaN,NaN,4.360614,29281.927038,...,NaN,NaN,2056.580500,1.637566,0.000000,0.299200,1726.940225,6.218422,87996.297000,0.238043
2021-01-01 15:04:00,NaN,NaN,0.000000,3.724216,NaN,NaN,NaN,NaN,2.620863,29281.734065,...,NaN,NaN,334.148800,1.637444,0.000000,0.299200,3590.968546,6.216047,73747.904000,0.238044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 13:15:00,748.782823,7.517841,13.863603,45.890622,0.000000,58.765350,6.830231,276.040637,0.657629,38021.177807,...,2946.396633,1.480779,280.840973,56.795526,54.215121,1.338541,921.038778,14.483248,18600.867465,0.667274
2023-01-31 13:16:00,1711.776069,7.517808,16.577327,45.890617,45.360064,58.765264,1.091403,276.041513,0.857525,38021.165182,...,6191.762224,1.480774,455.824680,56.795422,571.241028,1.338539,544.728049,14.483180,22391.441465,0.667273
2023-01-31 13:17:00,512.456285,7.517797,1.870275,45.890617,4.759460,58.765255,2.238450,276.043310,3.007150,38021.120921,...,1389.853643,1.480773,239.935366,56.795367,141.850043,1.338539,1961.725418,14.482937,22905.081298,0.667272


In [111]:
multidf_okx = convert_to_multi(single_okx, t_okx)
# multidf_okx.to_csv('multi_okx.csv')
multidf_okx

volume                                               \
                             okx                                                
                        APE_USDT AVAX_USDT    AXS_USDT   BNB_USDT    BTC_USDT   
timestamp                                                                       
2021-01-01 15:00:00          NaN       NaN    0.000000        NaN         NaN   
2021-01-01 15:01:00          NaN       NaN    0.000000        NaN         NaN   
2021-01-01 15:02:00          NaN       NaN  137.169200   3.725500         NaN   
2021-01-01 15:03:00          NaN       NaN   33.754300   3.724216         NaN   
2021-01-01 15:04:00          NaN       NaN    0.000000   3.724216         NaN   
...                          ...       ...         ...        ...         ...   
2023-01-31 13:15:00   748.782823  7.517841   13.863603  45.890622    0.000000   
2023-01-31 13:16:00  1711.776069  7.517808   16.577327  45.890617   45.360064   
2023-01-31 13:17:00   512.456285  7.517797    1.870275  45.890617    4.759460   
2023-01-31 13:18:00  1348.665222  7.517771    2.912640  45.890616   60.319632   
2023-01-31 13:19:00  2379.821350  7.517724   14.915358  45.890612  213.522092   

                                                                  \
                                                                   
                     DOGE_USDT   DOT_USDT    ETH_USDT   FTM_USDT   
timestamp                                                          
2021-01-01 15:00:00        NaN        NaN         NaN   2.567870   
2021-01-01 15:01:00        NaN        NaN         NaN   1.985712   
2021-01-01 15:02:00        NaN        NaN         NaN  35.688282   
2021-01-01 15:03:00        NaN        NaN         NaN   4.360614   
2021-01-01 15:04:00        NaN        NaN         NaN   2.620863   
...                        ...        ...         ...        ...   
2023-01-31 13:15:00  58.765350   6.830231  276.040637   0.657629   
2023-01-31 13:16:00  58.765264   1.091403  276.041513   0.857525   
2023-01-31 13:17:00  58.765255   2.238450  276.043310   3.007150   
2023-01-31 13:18:00  58.765140   1.299594  276.044354   2.210284   
2023-01-31 13:19:00  58.764734  19.849352  276.060367  22.001453   

                                   ...         vwap                         \
                                   ...          okx                          
                         GMT_USDT  ...     FTM_USDT  GMT_USDT    LINK_USDT   
timestamp                          ...                                       
2021-01-01 15:00:00  29319.650000  ...          NaN       NaN   374.301200   
2021-01-01 15:01:00  29314.678723  ...          NaN       NaN   590.031900   
2021-01-01 15:02:00  29282.948554  ...          NaN       NaN   954.226900   
2021-01-01 15:03:00  29281.927038  ...          NaN       NaN  2056.580500   
2021-01-01 15:04:00  29281.734065  ...          NaN       NaN   334.148800   
...                           ...  ...          ...       ...          ...   
2023-01-31 13:15:00  38021.177807  ...  2946.396633  1.480779   280.840973   
2023-01-31 13:16:00  38021.165182  ...  6191.762224  1.480774   455.824680   
2023-01-31 13:17:00  38021.120921  ...  1389.853643  1.480773   239.935366   
2023-01-31 13:18:00  38021.088400  ...   501.965245  1.480773   256.625593   
2023-01-31 13:19:00  38020.764852  ...  5087.802162  1.480769   409.438981   

                                                                              \
                                                                               
                    MATIC_USDT   NEAR_USDT SAND_USDT     SOL_USDT STORJ_USDT   
timestamp                                                                      
2021-01-01 15:00:00   1.642000    0.000000       NaN  3068.481619   6.225000   
2021-01-01 15:01:00   1.641388  177.490000  0.299200  1247.744360   6.223844   
2021-01-01 15:02:00   1.638708    0.000000  0.299200   831.909741   6.222253   
2021-01-01 15:03:00   1.637566    0.000000  0.299200  1726

### Binance Futures

In [110]:
single_binance_futures, t_binance_futures = create_single(df_binance_futures)
# single_binance_futures.to_csv('single_binance_futures.csv')
single_binance_futures

,volume-binance_futures:APE_USDT,vwap-binance_futures:APE_USDT,volume-binance_futures:AVAX_USDT,vwap-binance_futures:AVAX_USDT,volume-binance_futures:AXS_USDT,vwap-binance_futures:AXS_USDT,volume-binance_futures:BAKE_USDT,vwap-binance_futures:BAKE_USDT,volume-binance_futures:BNB_USDT,vwap-binance_futures:BNB_USDT,...,volume-binance_futures:SOL_USDT,vwap-binance_futures:SOL_USDT,volume-binance_futures:STORJ_USDT,vwap-binance_futures:STORJ_USDT,volume-binance_futures:UNFI_USDT,vwap-binance_futures:UNFI_USDT,volume-binance_futures:WAVES_USDT,vwap-binance_futures:WAVES_USDT,volume-binance_futures:XRP_USDT,vwap-binance_futures:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-09-08 17:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 17:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 17:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-18 23:56:00,37428.0,8.456447,5550.0,36.497122,15879.0,24.142576,12258.0,0.804090,15508.34,248.594339,...,15038.0,51.113558,32107.0,1.057372,575.4,7.324133,33136.3,8.991889,677541.4,0.669356
2022-12-18 23:57:00,36678.0,8.456439,3647.0,36.497113,21422.0,24.142546,3960.0,0.804090,3692.72,248.594342,...,24303.0,51.113494,30694.0,1.057372,464.1,7.324133,35116.0,8.991872,1238637.4,0.669356
2022-12-18 23:58:00,16696.0,8.456436,7472.0,36.497093,15383.0,24.142525,20330.0,0.804089,3891.48,248.594344,...,23563.0,51.113432,69130.0,1.057371,2097.1,7.324132,49625.4,8.991849,1234138.6,0.669355


In [114]:
multidf_binance_futures = convert_to_multi(single_binance_futures, t_binance_futures)
# multidf_binance_futures.to_csv("multi_binance_futures.csv")
multidf_binance_futures

volume                                         \
                    binance_futures                                          
                           APE_USDT AVAX_USDT AXS_USDT  BAKE_USDT BNB_USDT   
timestamp                                                                    
2019-09-08 17:57:00             NaN       NaN      NaN        NaN      NaN   
2019-09-08 17:58:00             NaN       NaN      NaN        NaN      NaN   
2019-09-08 17:59:00             NaN       NaN      NaN        NaN      NaN   
2019-09-08 18:00:00             NaN       NaN      NaN        NaN      NaN   
2019-09-08 18:01:00             NaN       NaN      NaN        NaN      NaN   
...                             ...       ...      ...        ...      ...   
2022-12-18 23:56:00         37428.0  8.456447   5550.0  36.497122  15879.0   
2022-12-18 23:57:00         36678.0  8.456439   3647.0  36.497113  21422.0   
2022-12-18 23:58:00         16696.0  8.456436   7472.0  36.497093  15383.0   
2022-12-18 23:59:00         60100.0  8.456423   3140.0  36.497084   2527.0   
2022-12-19 00:00:00        112388.0  8.456399  12174.0  36.497052  29370.0   

                                                                          ...  \
                                                                          ...   
                      BTC_BUSD  BTC_USDT  CRV_USDT  CTK_USDT   DOGE_USDT  ...   
timestamp                                                                 ...   
2019-09-08 17:57:00        NaN       NaN       NaN       NaN         NaN  ...   
2019-09-08 17:58:00        NaN       NaN       NaN       NaN         NaN  ...   
2019-09-08 17:59:00        NaN       NaN       NaN       NaN         NaN  ...   
2019-09-08 18:00:00        NaN       NaN       NaN       NaN         NaN  ...   
2019-09-08 18:01:00        NaN       NaN       NaN       NaN         NaN  ...   
...                        ...       ...       ...       ...         ...  ...   
2022-12-18 23:56:00  24.142576   12258.0  0.804090  15508.34  248.594339  ...   
2022-12-18 23:57:00  24.142546    3960.0  0.804090   3692.72  248.594342  ...   
2022-12-18 23:58:00  24.142525   20330.0  0.804089   3891.48  248.594344  ...   
2022-12-18 23:59:00  24.142521   17605.0  0.804089   1418.71  248.594345  ...   
2022-12-19 00:00:00  24.142480  152564.0  0.804087   9030.81  248.594352  ...   

                               vwap                                           \
                    binance_futures                                            
                         MATIC_USDT  NEAR_USDT  OGN_USDT RUNE_USDT SAND_USDT   
timestamp                                                                      
2019-09-08 17:57:00             NaN        NaN       NaN       NaN       NaN   
2019-09-08 17:58:00             NaN        NaN       NaN       NaN       NaN   
2019-09-08 17:59:00             NaN        NaN       NaN       NaN       NaN   
2019-09-08 18:00:00             NaN        NaN       NaN       NaN       NaN   
2019-09-08 18:01:00             NaN        NaN       NaN       NaN       NaN   
...                             ...        ...       ...       ...       ...   
2022-12-18 23:56:00         15038.0  51.113558   32107.0  1.057372     575.4   
2022-12-18 23:57:00         24303.0  51.113494   30694.0  1.057372     464.1   
2022-12-18 23:58:00         23563.0  51.113432   69130.0  1.057371    2097.1   
2022-12-18 23:59:00          3297.0  51.113423   79302.0  1.057369    1713.6   
2022-12-19 00:00:00         19258.0  51.113372  165224.0  1.057367    6906.6   

                                                                         
                                                                         
                     SOL_USDT STORJ_USDT UNFI_USDT WAVES_USDT  XRP_USDT  
timestamp                                                                
2019-09-08 17:57:00       NaN        NaN       NaN        NaN       NaN  
2019-09-08 17:58:00       NaN        NaN       NaN        NaN       NaN

## Binance Spot

In [115]:
single_binance_spot, t_binance_spot = create_single(df_binance_spot)
#single_binance_spot.to_csv('single_binance_spot.csv')
single_binance_spot 

,volume-binance_spot:APE_USDT,vwap-binance_spot:APE_USDT,volume-binance_spot:AVAX_USDT,vwap-binance_spot:AVAX_USDT,volume-binance_spot:AXS_USDT,vwap-binance_spot:AXS_USDT,volume-binance_spot:BAKE_USDT,vwap-binance_spot:BAKE_USDT,volume-binance_spot:BNB_USDT,vwap-binance_spot:BNB_USDT,...,volume-binance_spot:SOL_USDT,vwap-binance_spot:SOL_USDT,volume-binance_spot:STORJ_USDT,vwap-binance_spot:STORJ_USDT,volume-binance_spot:UNFI_USDT,vwap-binance_spot:UNFI_USDT,volume-binance_spot:WAVES_USDT,vwap-binance_spot:WAVES_USDT,volume-binance_spot:XRP_USDT,vwap-binance_spot:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,NaN,NaN,8358.48,3.153050,3419.80,0.537645,NaN,NaN,807.624,37.354150,...,481.32,1.507900,5830.18,0.290650,343.48,5.347700,82.50,6.198050,671035.5,0.219265
2021-01-01 00:01:00,NaN,NaN,3946.00,3.146668,7397.80,0.537515,NaN,NaN,2734.241,37.378274,...,1062.13,1.513268,21784.40,0.291044,68.49,5.347941,49.65,6.198614,977971.6,0.219728
2021-01-01 00:02:00,NaN,NaN,5531.79,3.141499,8846.40,0.536912,NaN,NaN,778.868,37.376044,...,423.25,1.513813,6071.42,0.291036,1824.99,5.328001,414.62,6.199854,438672.1,0.219846
2021-01-01 00:03:00,NaN,NaN,3586.61,3.139950,5425.00,0.536461,NaN,NaN,890.907,37.363865,...,2.28,1.513813,11454.74,0.290888,241.50,5.328327,113.45,6.200300,436956.9,0.219916
2021-01-01 00:04:00,NaN,NaN,3981.03,3.140162,1550.60,0.536309,NaN,NaN,483.711,37.356313,...,49.20,1.513811,11078.34,0.290723,308.63,5.327328,341.49,6.200386,457735.0,0.219989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 23:55:00,154.52,9.411818,599.82,43.936134,468.70,26.066194,255.1,1.478914,36.352,321.742836,...,7315.58,56.214752,964.00,1.358610,271.30,8.366264,281.75,10.713475,135505.0,0.735969
2023-01-31 23:56:00,264.84,9.411818,584.41,43.936128,5126.13,26.066163,816.3,1.478914,151.114,321.742835,...,1832.62,56.214736,4170.00,1.358610,1395.70,8.366261,976.73,10.713472,177663.0,0.735969
2023-01-31 23:57:00,287.58,9.411818,1333.10,43.936115,39.04,26.066163,0.0,1.478914,59.660,321.742835,...,798.39,56.214728,934.00,1.358610,60.60,8.366261,227.68,10.713472,197800.0,0.735969


In [116]:
multidf_binance_spot = convert_to_multi(single_binance_spot, t_binance_spot)
# multidf_binance_spot.to_csv('multi_binance_spot.csv')
multidf_binance_spot

volume                                         \
                    binance_spot                                          
                        APE_USDT AVAX_USDT AXS_USDT  BAKE_USDT BNB_USDT   
timestamp                                                                 
2021-01-01 00:00:00          NaN       NaN  8358.48   3.153050  3419.80   
2021-01-01 00:01:00          NaN       NaN  3946.00   3.146668  7397.80   
2021-01-01 00:02:00          NaN       NaN  5531.79   3.141499  8846.40   
2021-01-01 00:03:00          NaN       NaN  3586.61   3.139950  5425.00   
2021-01-01 00:04:00          NaN       NaN  3981.03   3.140162  1550.60   
...                          ...       ...      ...        ...      ...   
2023-01-31 23:55:00       154.52  9.411818   599.82  43.936134   468.70   
2023-01-31 23:56:00       264.84  9.411818   584.41  43.936128  5126.13   
2023-01-31 23:57:00       287.58  9.411818  1333.10  43.936115    39.04   
2023-01-31 23:58:00       887.80  9.411818   642.07  43.936109   494.47   
2023-01-31 23:59:00       404.81  9.411817   304.73  43.936106   621.26   

                                                                         ...  \
                                                                         ...   
                      BTC_BUSD BTC_USDT  CRV_USDT  CTK_USDT   DOGE_USDT  ...   
timestamp                                                                ...   
2021-01-01 00:00:00   0.537645      NaN       NaN   807.624   37.354150  ...   
2021-01-01 00:01:00   0.537515      NaN       NaN  2734.241   37.378274  ...   
2021-01-01 00:02:00   0.536912      NaN       NaN   778.868   37.376044  ...   
2021-01-01 00:03:00   0.536461      NaN       NaN   890.907   37.363865  ...   
2021-01-01 00:04:00   0.536309      NaN       NaN   483.711   37.356313  ...   
...                        ...      ...       ...       ...         ...  ...   
2023-01-31 23:55:00  26.066194    255.1  1.478914    36.352  321.742836  ...   
2023-01-31 23:56:00  26.066163    816.3  1.478914   151.114  321.742835  ...   
2023-01-31 23:57:00  26.066163      0.0  1.478914    59.660  321.742835  ...   
2023-01-31 23:58:00  26.066160   6072.7  1.478914    53.660  321.742834  ...   
2023-01-31 23:59:00  26.066156   1184.4  1.478914    17.317  321.742834  ...   

                            vwap                                           \
                    binance_spot                                            
                      MATIC_USDT  NEAR_USDT  OGN_USDT RUNE_USDT SAND_USDT   
timestamp                                                                   
2021-01-01 00:00:00       481.32   1.507900   5830.18  0.290650    343.48   
2021-01-01 00:01:00      1062.13   1.513268  21784.40  0.291044     68.49   
2021-01-01 00:02:00       423.25   1.513813   6071.42  0.291036   1824.99   
2021-01-01 00:03:00         2.28   1.513813  11454.74  0.290888    241.50   
2021-01-01 00:04:00        49.20   1.513811  11078.34  0.290723    308.63   
...                          ...        ...       ...       ...       ...   
2023-01-31 23:55:00      7315.58  56.214752    964.00  1.358610    271.30   
2023-01-31 23:56:00      1832.62  56.214736   4170.00  1.358610   1395.70   
2023-01-31 23:57:00       798.39  56.214728    934.00  1.358610     60.60   
2023-01-31 23:58:00      1653.47  56.214713    947.00  1.358610     30.10   
2023-01-31 23:59:00      1149.49  56.214703     75.00  1.358610     15.10   

                                                                          
                                                                          
                     SOL_USDT STORJ_USDT  UNFI_USDT WAVES_USDT  XRP_USDT  
timestamp                                                                 
2021-01-01 00:00:00  5.347700      82.50   6.198050   671035.5  0.219265  
2021-01-01 00:01:00  5.347941      49.65   6.198614   977971.6  0.219728  
2021-01-01 00:02:00  5.328001     414.62   6.199854   438672.1  0.219846  
2021-01-01 00:03:0

## Binance US

In [117]:
single_binance_us, t_binance_us = create_single(df_binance_us)
single_binance_us.to_csv('single_binance_us.csv')
single_binance_us

,volume-binanceus:APE_USDT,vwap-binanceus:APE_USDT,volume-binanceus:AVAX_USDT,vwap-binanceus:AVAX_USDT,volume-binanceus:AXS_USDT,vwap-binanceus:AXS_USDT,volume-binanceus:BNB_USDT,vwap-binanceus:BNB_USDT,volume-binanceus:BTC_BUSD,vwap-binanceus:BTC_BUSD,...,volume-binanceus:NEAR_USDT,vwap-binanceus:NEAR_USDT,volume-binanceus:OGN_USDT,vwap-binanceus:OGN_USDT,volume-binanceus:SAND_USDT,vwap-binanceus:SAND_USDT,volume-binanceus:SOL_USDT,vwap-binanceus:SOL_USDT,volume-binanceus:XRP_USDT,vwap-binanceus:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,8.03,37.352600,0.016564,28967.490000,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,649.1,0.219180
2021-01-01 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,17.78,37.371613,0.129987,29034.744841,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,10657.2,0.220009
2021-01-01 00:02:00,NaN,NaN,NaN,NaN,NaN,NaN,10.43,37.362572,0.012593,29035.266318,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.0,0.220009
2021-01-01 00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,0.00,37.362572,0.025558,29032.653630,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.0,0.220009
2021-01-01 00:04:00,NaN,NaN,NaN,NaN,NaN,NaN,0.00,37.362572,0.008773,29031.959243,...,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,0.0,0.220009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 23:55:00,0.0,9.78506,0.0,54.568026,0.0,58.213562,0.00,338.953264,0.000000,33115.363506,...,0.0,8.139902,0.0,0.420419,0.0,0.663024,11.75,64.860428,11914.6,0.275762
2023-01-31 23:56:00,0.0,9.78506,0.0,54.568026,0.0,58.213562,0.00,338.953264,0.000000,33115.363506,...,0.0,8.139902,0.0,0.420419,0.0,0.663024,117.93,64.860364,11914.6,0.275762
2023-01-31 23:57:00,0.0,9.78506,0.0,54.568026,0.0,58.213562,0.00,338.953264,0.000000,33115.363506,...,0.0,8.139902,0.0,0.420419,0.0,0.663024,0.00,64.860364,11914.6,0.275762


In [118]:
multidf_binance_us = convert_to_multi(single_binance_us, t_binance_us)
multidf_binance_us.to_csv('multi_binance_us.csv')
multidf_binance_us

volume                                         \
                    binanceus                                          
                     APE_USDT AVAX_USDT AXS_USDT   BNB_USDT BTC_BUSD   
timestamp                                                              
2021-01-01 00:00:00       NaN       NaN      NaN        NaN      NaN   
2021-01-01 00:01:00       NaN       NaN      NaN        NaN      NaN   
2021-01-01 00:02:00       NaN       NaN      NaN        NaN      NaN   
2021-01-01 00:03:00       NaN       NaN      NaN        NaN      NaN   
2021-01-01 00:04:00       NaN       NaN      NaN        NaN      NaN   
...                       ...       ...      ...        ...      ...   
2023-01-31 23:55:00       0.0   9.78506      0.0  54.568026      0.0   
2023-01-31 23:56:00       0.0   9.78506      0.0  54.568026      0.0   
2023-01-31 23:57:00       0.0   9.78506      0.0  54.568026      0.0   
2023-01-31 23:58:00       0.0   9.78506      0.0  54.568026      0.0   
2023-01-31 23:59:00       0.0   9.78506      0.0  54.568026      0.0   

                                                                             \
                                                                              
                      BTC_USDT CRV_USDT   DOGE_USDT  DOT_USDT      ETH_BUSD   
timestamp                                                                     
2021-01-01 00:00:00        NaN     8.03   37.352600  0.016564  28967.490000   
2021-01-01 00:01:00        NaN    17.78   37.371613  0.129987  29034.744841   
2021-01-01 00:02:00        NaN    10.43   37.362572  0.012593  29035.266318   
2021-01-01 00:03:00        NaN     0.00   37.362572  0.025558  29032.653630   
2021-01-01 00:04:00        NaN     0.00   37.362572  0.008773  29031.959243   
...                        ...      ...         ...       ...           ...   
2023-01-31 23:55:00  58.213562     0.00  338.953264  0.000000  33115.363506   
2023-01-31 23:56:00  58.213562     0.00  338.953264  0.000000  33115.363506   
2023-01-31 23:57:00  58.213562     0.00  338.953264  0.000000  33115.363506   
2023-01-31 23:58:00  58.213562     0.20  338.953263  0.000000  33115.363506   
2023-01-31 23:59:00  58.213562     0.00  338.953263  0.000000  33115.363506   

                     ...      vwap                                         \
                     ... binanceus                                          
                     ...  DOT_USDT  ETH_BUSD ETH_USDT  FTM_USDT LINK_USDT   
timestamp            ...                                                    
2021-01-01 00:00:00  ...       NaN       NaN      NaN       NaN       NaN   
2021-01-01 00:01:00  ...       NaN       NaN      NaN       NaN       NaN   
2021-01-01 00:02:00  ...       NaN       NaN      NaN       NaN       NaN   
2021-01-01 00:03:00  ...       NaN       NaN      NaN       NaN       NaN   
2021-01-01 00:04:00  ...       NaN       NaN      NaN       NaN       NaN   
...                  ...       ...       ...      ...       ...       ...   
2023-01-31 23:55:00  ...       0.0  8.139902      0.0  0.420419       0.0   
2023-01-31 23:56:00  ...       0.0  8.139902      0.0  0.420419       0.0   
2023-01-31 23:57:00  ...       0.0  8.139902      0.0  0.420419       0.0   
2023-01-31 23:58:00  ...       0.0  8.139902      0.0  0.420419       0.0   
2023-01-31 23:59:00  ...       0.0  8.139902      0.0  0.420419       0.0   

                                                                      
                                                                      
                    NEAR_USDT OGN_USDT  SAND_USDT SOL_USDT  XRP_USDT  
timestamp                                                             
2021-01-01 00:00:00       NaN     0.00        NaN    649.1  0.219180  
2021-01-01 00:01:00       NaN     0.00        NaN  10657.2  0.220009  
2021-01-01 00:02:00       NaN     0.00        NaN      0.0  0.220009  
2021-01-01 00:03:00       NaN     0.00        NaN      0.0  0.220009  
2021-01-01 00:04:00       NaN   

# now to make the single df with all exchanges
using merge as of, i run into the problem either for ex. if i'm merging single_okx with single_binance_futures, then i can either keep all of the timestamps of one of the dataframes, but if they're nonoverlapping time intervals then this is an issue

In [119]:
merged = pd.merge(single_okx, single_binance_futures, on='timestamp', how='outer').sort_index()
merged = pd.merge(merged, single_binance_spot, on='timestamp', how='outer').sort_index()
merged = pd.merge(merged, single_binance_us, on='timestamp', how='outer').sort_index()

In [120]:
merged

,volume-okx:APE_USDT,vwap-okx:APE_USDT,volume-okx:AVAX_USDT,vwap-okx:AVAX_USDT,volume-okx:AXS_USDT,vwap-okx:AXS_USDT,volume-okx:BNB_USDT,vwap-okx:BNB_USDT,volume-okx:BTC_USDT,vwap-okx:BTC_USDT,...,volume-binanceus:NEAR_USDT,vwap-binanceus:NEAR_USDT,volume-binanceus:OGN_USDT,vwap-binanceus:OGN_USDT,volume-binanceus:SAND_USDT,vwap-binanceus:SAND_USDT,volume-binanceus:SOL_USDT,vwap-binanceus:SOL_USDT,volume-binanceus:XRP_USDT,vwap-binanceus:XRP_USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2019-09-08 17:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 17:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 17:59:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-09-08 18:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-31 23:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,8.139902,0.0,0.420419,0.0,0.663024,11.75,64.860428,11914.6,0.275762
2023-01-31 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,8.139902,0.0,0.420419,0.0,0.663024,117.93,64.860364,11914.6,0.275762
2023-01-31 23:57:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,8.139902,0.0,0.420419,0.0,0.663024,0.00,64.860364,11914.6,0.275762


In [121]:
merged.to_csv('merged.csv')

In [122]:
t_merged = pd.DataFrame(merged.index)
t_merged

,timestamp
0,2019-09-08 17:57:00
1,2019-09-08 17:58:00
2,2019-09-08 17:59:00
3,2019-09-08 18:00:00
4,2019-09-08 18:01:00
...,...
1787397,2023-01-31 23:55:00
1787398,2023-01-31 23:56:00
1787399,2023-01-31 23:57:00
1787400,2023-01-31 23:58:00


In [123]:
multi_merged = convert_to_multi(merged, t_merged)

In [124]:
multi_merged

volume                                                \
                         okx                                                 
                    APE_USDT AVAX_USDT AXS_USDT BNB_USDT BTC_BUSD BTC_USDT   
timestamp                                                                    
2019-09-08 17:57:00      NaN       NaN      NaN      NaN      NaN      NaN   
2019-09-08 17:58:00      NaN       NaN      NaN      NaN      NaN      NaN   
2019-09-08 17:59:00      NaN       NaN      NaN      NaN      NaN      NaN   
2019-09-08 18:00:00      NaN       NaN      NaN      NaN      NaN      NaN   
2019-09-08 18:01:00      NaN       NaN      NaN      NaN      NaN      NaN   
...                      ...       ...      ...      ...      ...      ...   
2023-01-31 23:55:00      NaN       NaN      NaN      NaN      NaN      NaN   
2023-01-31 23:56:00      NaN       NaN      NaN      NaN      NaN      NaN   
2023-01-31 23:57:00      NaN       NaN      NaN      NaN      NaN      NaN   
2023-01-31 23:58:00      NaN       NaN      NaN      NaN      NaN      NaN   
2023-01-31 23:59:00      NaN       NaN      NaN      NaN      NaN      NaN   

                                                          ...       vwap  \
                                                          ...  binanceus   
                    CRV_USDT DOGE_USDT DOT_USDT ETH_BUSD  ... MATIC_USDT   
timestamp                                                 ...              
2019-09-08 17:57:00      NaN       NaN      NaN      NaN  ...        NaN   
2019-09-08 17:58:00      NaN       NaN      NaN      NaN  ...        NaN   
2019-09-08 17:59:00      NaN       NaN      NaN      NaN  ...        NaN   
2019-09-08 18:00:00      NaN       NaN      NaN      NaN  ...        NaN   
2019-09-08 18:01:00      NaN       NaN      NaN      NaN  ...        NaN   
...                      ...       ...      ...      ...  ...        ...   
2023-01-31 23:55:00      NaN       NaN      NaN      NaN  ...        0.0   
2023-01-31 23:56:00      NaN       NaN      NaN      NaN  ...        0.0   
2023-01-31 23:57:00      NaN       NaN      NaN      NaN  ...        0.0   
2023-01-31 23:58:00      NaN       NaN      NaN      NaN  ...        0.0   
2023-01-31 23:59:00      NaN       NaN      NaN      NaN  ...        0.0   

                                                                      \
                                                                       
                    NEAR_USDT OGN_USDT RUNE_USDT SAND_USDT  SOL_USDT   
timestamp                                                              
2019-09-08 17:57:00       NaN      NaN       NaN       NaN       NaN   
2019-09-08 17:58:00       NaN      NaN       NaN       NaN       NaN   
2019-09-08 17:59:00       NaN      NaN       NaN       NaN       NaN   
2019-09-08 18:00:00       NaN      NaN       NaN       NaN       NaN   
2019-09-08 18:01:00       NaN      NaN       NaN       NaN       NaN   
...                       ...      ...       ...       ...       ...   
2023-01-31 23:55:00  8.139902      0.0  0.420419       0.0  0.663024   
2023-01-31 23:56:00  8.139902      0.0  0.420419       0.0  0.663024   
2023-01-31 23:57:00  8.139902      0.0  0.420419       0.0  0.663024   
2023-01-31 23:58:00  8.139902      0.0  0.420419       0.0  0.663024   
2023-01-31 23:59:00  8.139902      0.0  0.420419       0.0  0.663024   

                                                                
                                                                
                    STORJ_USDT  UNFI_USDT WAVES_USDT  XRP_USDT  
timestamp                                                       
2019-09-08 17:57:00        NaN        NaN        NaN       NaN  
2019-09-08 17:58:00        NaN        NaN        NaN       NaN  
2019-09-08 17:59:00        NaN        NaN        NaN       NaN  
2019-09-08 18:00:00        NaN        NaN        NaN       NaN  
2019-09-08 18:01:00        NaN        NaN        NaN       NaN  
...                        ...        ...   

In [129]:
multi_merged.to_csv('multi_merged.csv')

In [128]:
multi_merged.columns.levels[1]

Index(['binance_futures', 'binance_spot', 'binanceus', 'okx'], dtype='object')